# Why Non-Parametric Tests?
- Some experiments yield response measurements that defy exact quantification.
    - e.g. it is impossible to make statements such as “teacher A is twice as good as teacher B.”
- Insufficient knowledge to estimate if the population(s) fulfil the assumptions that specified by parametric tests.

In [2]:
import pandas as pd
import numpy as np
import pingouin as pg
from scipy import stats
from statsmodels.stats.descriptivestats import sign_test
from pydataset import data

In [3]:
df = data('iris')

# Paired Sign Test 
- Number of positive differences, M where $D_i = X_i − Y_i$ follows a binomial distribution, $Bin(n, p)$.
- Test statistic: M = number of positive differences where $D_i = X_i − Y_i$
- $H_0$: $p=0.5$
- $H_a$: $p\neq0.5$

In [30]:
def sign_test(x, y, tail='two-sided', p0=0.5):
    M = (x>y).sum()
    left_p = stats.binom.cdf(M, len(x), p0) # P(X<=x)
    if tail=='greater':
        return M, 1-left_p
    if tail=='less':
        return M, left_p
    else:
        return M, (left_p if left_p<0.5 else 1-left_p) * 2

In [32]:
M, pval = sign_test(
    x = df.loc[df['Species']=='setosa','Sepal.Width'].values, 
    y = df.loc[df['Species']=='versicolor','Sepal.Width'].values,
    tail='two-sided', 
    p0=0.5
)
print("M:", M, "p-value:", pval)

M: 43 p-value: 3.243740565039843e-08


# Wilcoxon Signed-Rank Test (Paired Samples)

- Assumption: Independent random samples have been drawn from 2 populations that differ only in location.
- $H_0$: The population distributions for the X’s and Y ’s are identical.
- $H_a$: 
    - The two population distributions differ in location (two-tailed),
    - The population relative frequency distribution for the X’s is shifted to the right/left of that for the Y ’s (one-tailed).
    
- Procedure: The absolute values of the pair differences are ranked, 
    - W- indicates the sum of rank of the negative differences
    - W+ indicates the sum of rank of the positive differences
Test statistic:
1. For a two-tailed test, use $W = min(W+, W−)$
2. For a one-tailed test (to detect the one-tailed alternative just given), use the rank sum T− of the negative differences.


In [33]:
pg.wilcoxon(
    x=df.loc[df['Species']=='setosa','Sepal.Length'], 
    y=df.loc[df['Species']=='versicolor','Sepal.Length'],
    tail='two-sided'
)

,W-val,tail,p-val,RBC,CLES
Wilcoxon,19.0,two-sided,3.586548e-09,-0.96898,0.0674


# Mann–Whitney U Test (Independent Samples)
- Assumption: Independent random samples have been drawn from 2 populations that differ only in location.
- Equivalent to Wilcoxon Rank-sum Test
    - Procedure: Rank $n_1 + n_2 = n$ observations in terms of magnitude
    - Rationale: The expected rank sums for the samples should be proportional to the sample sizes n1 and n2. If the observations in one population tended to be larger, the rank sum in its sample is larger than the expected rank sum.
- Test statistic: $U = n_1n_2 + \frac{n_1(n_1 + 1)}{2}-W$, where $W$ is the smaller rank sum.
- $H_0$: The population distributions for the X’s and Y ’s are identical.
- $H_a$: 
    - The two population distributions differ in location (two-tailed),
    - The population relative frequency distribution for the X’s is shifted to the right/left of that for the Y ’s (one-tailed).

In [85]:
pg.mwu(
    x = df.loc[df['Species']=='setosa','Sepal.Length'], 
    y = df.loc[df['Species']=='versicolor','Sepal.Length'],
    tail = 'two-sided'
)

,U-val,tail,p-val,RBC,CLES
MWU,168.5,two-sided,8.345827e-14,0.8652,0.0674


# Kruskal–Wallis Test
- Non-parametric version of one-way ANOVA.
- Rationale: Rank analogue of SST $V = \sum_{i=1}^{k}n_i(\bar{R_i}-\bar{R})$
    - $\bar{R_i} = R_i/n_i$ denotes average rank in a category.
    - $\bar{R}$ = (sum of first n int)/n = $[n(n+1)/2]/n = (n+1)/2$ denotes the overall overage rank.
    - large $V$ indicates that $H_0$ should be rejected. 
- Test statistic: A transformed version of V, when each n_i is greater than 5, it can be approximated using $\chi^2$ with $k-1$ dof.
- $H_0$: The *k* population distributions are identical.
- $H_a$: At least two of the population distributions differ in location.

In [10]:
pg.kruskal(
    data=df, 
    dv='Sepal.Length', 
    between='Species'
)

,Source,ddof1,H,p-unc
Kruskal,Species,2,96.937436,8.918734e-22


## Post-hoc Test: Pairwise Mann-Whitney U Test
- with adjusted *p*-values

In [3]:
df_mixed_anova = pg.read_dataset('mixed_anova.csv')

In [4]:
pg.pairwise_ttests(
    data=df_mixed_anova,
    dv='Scores', 
    between=['Time'],    
    padjust='holm',
    parametric=False
)

,Contrast,A,B,Paired,Parametric,U-val,Tail,p-unc,p-corr,p-adjust,hedges
0,Time,August,January,False,False,1495.0,two-sided,0.109996,0.219992,holm,-0.327583
1,Time,August,June,False,False,1280.0,two-sided,0.006398,0.019193,holm,-0.482547
2,Time,January,June,False,False,1545.0,two-sided,0.181622,0.219992,holm,-0.169520


# Friedman Test (Randomized Block Design)

- Procedure: Let we have *b* blocks and *k* treatments, rank the observations from *k* treatments within each block.
    - $\bar{R_i} = \frac{\sum_{j=1}^{b}R_{ij}}{b}$ denotes average ranks of treatment *i*
    - $\bar{R} = \frac{bk(k+1)/2}{bk} = (k+1)/2$ denotes the overall overage rank.
    - $W = b\sum_{i=1}^{k}(\bar{R_i}-\bar{R})$ denotes the rank analogue of SST, large $W$ indicates $H_0$ should be rejected. 
    
- Test statistic: A transformed version of W, when either $b$ or $k$ is greater than 5, it can be approximated using $\chi^2$ with $k-1$ dof.
- $H_0$: The distributions of *k* distributions are identical.
- $H_a$: At least two of the treatments differ in locations of distributions.

In [ ]:
df['sepal_width_atleast3_1'] = df['Sepal.Width']>=3.1

In [11]:
pg.friedman(
    data=df, 
    dv='Sepal.Length', # response
    within='Species', # block
    subject='sepal_width_atleast3_1' # treatment
)

,Source,W,ddof1,Q,p-unc
Friedman,Species,1.0,2,4.0,0.135335


## Post-hoc Test: Pairwise Wilcoxon Signed-Rank Test
- with adjusted *p*-values

In [9]:
pg.pairwise_ttests(
    data=df_mixed_anova,
    dv='Scores', 
    between=['Time'], 
    subject='Subject',
    padjust='holm',
    parametric=False
)

,Contrast,A,B,Paired,Parametric,U-val,Tail,p-unc,p-corr,p-adjust,hedges
0,Time,August,January,False,False,1495.0,two-sided,0.109996,0.219992,holm,-0.327583
1,Time,August,June,False,False,1280.0,two-sided,0.006398,0.019193,holm,-0.482547
2,Time,January,June,False,False,1545.0,two-sided,0.181622,0.219992,holm,-0.169520
